In [1]:
import openpyxl
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import range_boundaries, get_column_letter
#from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import PatternFill
#https://stackoverflow.com/questions/72746988/merge-cells-using-python
from openpyxl import load_workbook
import xlwings as xw
from copy import copy
import os
import re

In [ ]:
###Dodelat:
###dplnovani/odstranovani sloupcu. vsude jsou tři sloupce
###prejmenovat nazvy listu, aby sli poporade. Např. ROM Ad Impact m
###Poresit ad impat m - aby se pro kazdou verzi tabulek zobrazili všechny reklamy v třídění
#VYresit tri desetinná místa u vázeneho Countu a Valid N

In [2]:
def merge_tabs (countries, type_tabs, adresa_tabs):
    
    for country in countries:
        for type_tab in type_tabs:
            input_tabs = f"{adresa_tabs}\\{country}\\{type_tab}\\formated\\to_format_.xlsx"
            formated_tabs = f"{adresa_tabs}\\{country}\\{type_tab}\\formated\\formated_.xlsx"
            output_tabs = f"{adresa_tabs}\\{country}\\{type_tab}\\formated\\output_.xlsx"

            ###První část
            ##odformátování tabulek a složení jednotlivých listů s tabulkami do jednoho souboru
            counter = 0

            tabs_to_format = f"{adresa_tabs}\\{country}\\{type_tab}"
            files = []
            filenames = [f"{tabs_to_format}\\{file}" for file in os.listdir(tabs_to_format) if file.lower().endswith(".xlsx")]
            if len(filenames) == 0:
                continue
            else:
                pass


            for index, filename in enumerate(filenames):
                files.append(filename.split("\\")[-1].split(".")[0])
                #print(files)

            column_num = 1
            header = False

            # Create a new workbook
            new_wb = openpyxl.Workbook() 

            # Read each workbook, and copy each column, cell by cell, to the new spreadsheet
            for index, filename in enumerate(filenames):
                #print(filename)
                wb = openpyxl.load_workbook(filename)
                ws = wb.active

                # Create a new workbook
                #new_wb = openpyxl.Workbook()
                #new_ws = new_wb.active
                #https://stackoverflow.com/questions/26328398/how-to-write-two-sheets-in-a-single-workbook-at-the-same-time-using-openpyxl
                new_ws = new_wb.create_sheet()
                new_ws.title = files[index]

                counter += 1

                # The min and max columns of every MergedCellRange needs to be shifted
                #print(filename)
                #print(min_col_2,":", min_row_2,":", max_col_2,":", max_row_2)
                for range_ in ws.merged_cells.ranges:
                    #print(range_)
                    min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
                    min_col = get_column_letter(min_col)
                    max_col = get_column_letter(max_col)
                    min_row = min_row
                    max_row = max_row
                    #print(min_col,":", min_row,":", max_col,":", max_row)
                    new_ws.merged_cells.add(f"{min_col}{min_row}:{max_col}{max_row}")



                for row in ws.iter_rows():
                    for cell in row:
                        #rounding
                        #print(cell.value)
                        if type(cell.value) == float:
                            cell.value = round(cell.value, 3)
                        new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
                            #print(cell.value)

                #Hiding gridlines     
                new_ws.sheet_view.showGridLines = False
                #smazani automaticky generovaného sheetu:
                if counter == 1:
                    del new_wb['Sheet']
                # Save the new workbook to disk
                joined_tab = input_tabs
                #print(dir(new_wb.save())
                new_wb.save(joined_tab)
                wb.close()
                new_wb.close()

                # Launch the new spreadsheet
                #os.startfile(new_filename)

                #break        

            ########
            ###Druhá část

            def find_coord_formated_tabs (ws):
                counter = 0
                for row in ws.iter_rows():
                    counter += 1
                    for cell in row:
                        if cell.fill.start_color.index == 22:
                            return counter

            with xw.App(visible=False) as app:
                wb_xlwing = xw.Book(input_tabs)   
                macro_vba_formattabs_beta = app.macro("'c:\\Users\\jan.petr\\Provozni\\excel_tab_copying\\CTables 4.2.35.xlsm'!a_FORMATTABS_beta")
                #macro_vba_sig_95 = app.macro("'c:\\Users\\jan.petr\\Provozni\\excel_tab_copying\\CTables 4.2.35.xlsm'!b_sig_min_run")
                for sheet in wb_xlwing.sheets:
                    sheet.activate()
                    macro_vba_formattabs_beta()
                wb_xlwing.save(formated_tabs)
                wb_xlwing.close()

            wb = openpyxl.load_workbook(formated_tabs)

            with xw.App(visible=False) as app:
                counter = 0
                width_list = []
                wb_xlwing = xw.Book(formated_tabs)
                for filename in wb.sheetnames:
                    counter += 1
                    #print(filename)
                    ws = wb[filename]
                    ws_xlwing=wb_xlwing.sheets[filename]
                    if counter == 1:
                        max_row = find_coord_formated_tabs (ws)
                        freezed = ws.freeze_panes
                    for col in range(ws.min_column, ws.max_column + 1):
                        width_to_append = ws_xlwing.range(str(get_column_letter(col)+"1")).column_width
                        if counter == 1:
                            width_list.append(width_to_append)
                        else:
                            if width_list[col - 1] < width_to_append:
                                width_list[col - 1] = width_to_append


            #print(width_list)

            with xw.App(visible=False) as app:
                wb_init = xw.Book(formated_tabs)
                wb_res = xw.Book()
                ws_res = wb_res.sheets[0]

                for index,ws_init in enumerate (wb_init.sheets):
                    if index == 0:
                        ws_init.used_range.copy()
                    else:
                        ws_init.used_range[max_row - 1:,:].copy()
                    ws_res.used_range[-1:,:].offset(row_offset=1).paste()
                #ws_res["1:1"].delete() # This is just to delete the first row, which is empty.
                for col in range(ws.min_column, ws.max_column + 1):
                    ws_res.range(str(get_column_letter(col)+"1")).column_width = width_list[col-1]
                #přizplsobí výšku bunek
                ws_res.autofit(axis="rows")
                #uloži
                wb_res.save(output_tabs)
                wb_res.close(); wb_init.close()


            #Ukotveni pricek
            new_wb = openpyxl.load_workbook(output_tabs)
            new_ws = new_wb.active
            #https://stackoverflow.com/questions/48752276/trying-to-unfreeze-panes-with-openpyxl-causes-damages
            new_ws.sheet_view.topLeftCell = 'A1'
            new_ws.freeze_panes = freezed
            #print(new_ws.freeze_panes)

            #Hiding gridlines
            new_ws.sheet_view.showGridLines = False

            new_wb.save(output_tabs)
            print("Done: ",country, " - ",type_tab)

In [31]:

def merge_to_final (countries, type_tabs, adresa_tabs, final_tabs):
    for country in countries:
        filenames = []
        files = []
        for type_tab in type_tabs:
            tabs_to_format = f"{adresa_tabs}\\{country}\\{type_tab}\\formated\\output_.xlsx"
            #filenames.append([f"{tabs_to_format}\\{file}" for file in os.listdir(tabs_to_format) if file.lower().endswith("combined_.xlsx")][0])
            filenames.append(tabs_to_format)
            #files.append(type_tab.split("1_")[1])
            files.append(type_tab.split(re.match(r"^[0-9]_", type_tab).group(0))[1])

        #print(filenames)
        #print(files)
        #print(len(filenames))
        #print(len(files))

        # Create a new workbook
        new_wb_xlwing = xw.Book()


        # Read each workbook, and copy each column, cell by cell, to the new spreadsheet
        for filename, file in zip(filenames, files):
            try:
                wb_xlwing = xw.Book(filename)
                wb_xlwing.sheets[0].name = file
                wb_xlwing.sheets[0].api.Copy(None, After=new_wb_xlwing.sheets[0].api)
                #wb_xlwing.sheets[0].api.Copy(Before=new_wb_xlwing.sheets[0].api)
                wb_xlwing.close()
                print(file, " Country:", country)
            except FileNotFoundError:
                pass
        try:
            ws_to_del=[sheet.name for sheet in new_wb_xlwing.sheets if sheet.name.lower().startswith("list")]
            if len(ws_to_del) > 0:
                for sheet in ws_to_del:
                    new_wb_xlwing.sheets[sheet].delete()
        # přeskočení chyby: com_error: https://stackoverflow.com/questions/25053785/how-to-catch-these-exceptions-individually
        except BaseException as e:
            print(e.args)
        
        app = xw.apps.active
        new_wb_xlwing.save(f"{adresa_tabs}\\{country}\\{final_tabs}\\final_{country}.xlsx")
        #new_wb_xlwing.close()
        app.kill()


In [4]:
countries = ["MNE", "BLG", "SRB", "ROM"]
type_tabs = ["1_development", "2_brand_tabs_q", "3_ad_impact_m", "5_brand_tabs_m", "6_ad_tabs_m", "7_ad_tabs_q"]
adresa_tabs = r"c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy"
final_tabs = "0_final_tabs"


In [5]:
##Zformátování a sloučení tabulek za jeden typ 
merge_tabs (countries = countries, type_tabs = type_tabs, adresa_tabs = adresa_tabs)

Done:  MNE  -  1_development
Done:  MNE  -  2_brand_tabs_q
Done:  MNE  -  3_ad_impact_m
Done:  MNE  -  5_brand_tabs_m
Done:  BLG  -  1_development
Done:  BLG  -  2_brand_tabs_q
Done:  BLG  -  3_ad_impact_m
Done:  BLG  -  5_brand_tabs_m
Done:  BLG  -  6_ad_tabs_m
Done:  BLG  -  7_ad_tabs_q
Done:  SRB  -  1_development
Done:  SRB  -  2_brand_tabs_q
Done:  SRB  -  3_ad_impact_m
Done:  SRB  -  5_brand_tabs_m
Done:  SRB  -  6_ad_tabs_m
Done:  SRB  -  7_ad_tabs_q
Done:  ROM  -  1_development
Done:  ROM  -  2_brand_tabs_q
Done:  ROM  -  3_ad_impact_m


com_error: (-2147352567, 'Došlo k výjimce.', (0, None, None, None, 0, -2146788248), None)

In [6]:
countries = ["ROM"]
type_tabs = ["5_brand_tabs_m", "6_ad_tabs_m", "7_ad_tabs_q"]
merge_tabs (countries = countries, type_tabs = type_tabs, adresa_tabs = adresa_tabs)

Done:  ROM  -  5_brand_tabs_m
Done:  ROM  -  6_ad_tabs_m
Done:  ROM  -  7_ad_tabs_q


In [33]:
#countries = ["CRO"]
#type_tabs = ["1_development", "2_brand_tabs_q", "3_ad_impact_m", "5_brand_tabs_m", "6_ad_tabs_m", "7_ad_tabs_q"]
#merge_tabs (countries = ["CRO"], type_tabs = type_tabs, adresa_tabs = adresa_tabs)

##spojeni jednotlivých tabulek do jednoho souboru za každou zemi
countries = ["BLG", "MNE", "SRB", "ROM", "CRO"]
type_tabs = ["1_development", "2_brand_tabs_q", "3_ad_impact_m", "5_brand_tabs_m", "6_ad_tabs_m", "7_ad_tabs_q"]
merge_to_final (countries=countries, type_tabs=type_tabs, adresa_tabs=adresa_tabs, final_tabs=final_tabs)
print("final BLG MNE SRB ROM CRO finnished")
countries = ["HUN"]
type_tabs = ["1_development", "2_brand_tabs_q", "3_ad_impact_m", "5_brand_tabs_m", "6_ad_tabs_m", "7_ad_tabs_q"]
print("lets go to the final HUN")
merge_tabs (countries = countries, type_tabs = type_tabs, adresa_tabs = adresa_tabs)

merge_to_final (countries=countries, type_tabs=type_tabs, adresa_tabs=adresa_tabs, final_tabs=final_tabs)

development  Country: BLG
brand_tabs_q  Country: BLG
ad_impact_m  Country: BLG
brand_tabs_m  Country: BLG
ad_tabs_m  Country: BLG
ad_tabs_q  Country: BLG
development  Country: MNE
brand_tabs_q  Country: MNE
ad_impact_m  Country: MNE
brand_tabs_m  Country: MNE
development  Country: SRB
brand_tabs_q  Country: SRB
ad_impact_m  Country: SRB
brand_tabs_m  Country: SRB
ad_tabs_m  Country: SRB
ad_tabs_q  Country: SRB
development  Country: ROM
brand_tabs_q  Country: ROM
ad_impact_m  Country: ROM
brand_tabs_m  Country: ROM
ad_tabs_m  Country: ROM
ad_tabs_q  Country: ROM
development  Country: CRO
brand_tabs_q  Country: CRO
ad_impact_m  Country: CRO
brand_tabs_m  Country: CRO
ad_tabs_m  Country: CRO
ad_tabs_q  Country: CRO
final BLG MNE SRB ROM CRO finnished
lets go to the final HUN
Done:  HUN  -  1_development
Done:  HUN  -  2_brand_tabs_q
Done:  HUN  -  3_ad_impact_m
Done:  HUN  -  5_brand_tabs_m
Done:  HUN  -  6_ad_tabs_m
Done:  HUN  -  7_ad_tabs_q
development  Country: HUN
brand_tabs_q  Count

In [53]:
##Sloučení všech typů tabulek do jedněch
merge_to_final (countries=countries, type_tabs=type_tabs, adresa_tabs=adresa_tabs, final_tabs=final_tabs)

development


In [25]:
jmeno = "2_ad_impact_m"

jetam = re.match(r"^[0-9]_", jmeno)
#print(dir(jetam))
#print(jetam.group(1))

print(re.match(r"^[0-9]_", jmeno).group(0))

2_


In [ ]:
    with xw.App(visible=False) as app:
        wb_xlwing = xw.Book(filename)
        #wb_xlwing.sheets[0].name = file
        #ws_xlwing = wb_xlwing.sheets[0]
        #print(ws_xlwing)
        #new_wb_xlwing.sheets.add(wb_xlwing.sheets[0].name)
        #wb_xlwing.sheets[0].copy(after=new_wb_xlwing.sheets[0])
        wb_xlwing.sheets[0].api.Copy(None, After=new_wb_xlwing.sheets[0].api)
        #print(new_wb_xlwing.sheets)
        new_wb_xlwing.save(f"{adresa_tabs}\\{country}\\{final_tabs}\\final.xlsx")
    print(file)

#def merge_to_final (countries, type_tabs, adresa_tabs, final_tabs):
    
    

In [7]:
print(len(filenames))

5


## Bere syrové tabulky přímo z spss.
   - načte tabulku, překopíruje do nové tabulky bez spss naformátování, aby se pak dali spustit format a signifikance
   - zkusíme rovnou spustet format a signifikance

In [37]:
counter = 0
    
adresa_tabs = r"c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m"
files = []
filenames = [f"{adresa_tabs}\\{file}" for file in os.listdir(adresa_tabs) if file.lower().endswith(".xlsx")]


for index, filename in enumerate(filenames):
    files.append(filename.split("\\")[-1].split(".")[0])
    print(files)
    #print(index)
#print(list(filenames))
# column_num is the next column number to be written to in the new spreadsheet
column_num = 1
header = False

# Create a new workbook
new_wb = openpyxl.Workbook() 


# Read each workbook, and copy each column, cell by cell, to the new spreadsheet
for index, filename in enumerate(filenames):
    #print(filename)
    wb = openpyxl.load_workbook(filename)
    ws = wb.active
    
    # Create a new workbook
    #new_wb = openpyxl.Workbook()
    #new_ws = new_wb.active
    #https://stackoverflow.com/questions/26328398/how-to-write-two-sheets-in-a-single-workbook-at-the-same-time-using-openpyxl
    new_ws = new_wb.create_sheet()
    new_ws.title = files[index]
    
    counter += 1
        
    # The min and max columns of every MergedCellRange needs to be shifted
    print(filename)
    #print(min_col_2,":", min_row_2,":", max_col_2,":", max_row_2)
    for range_ in ws.merged_cells.ranges:
        #print(range_)
        min_col, min_row, max_col, max_row = range_boundaries(range_.coord)
        min_col = get_column_letter(min_col)
        max_col = get_column_letter(max_col)
        min_row = min_row
        max_row = max_row
        #print(min_col,":", min_row,":", max_col,":", max_row)
        new_ws.merged_cells.add(f"{min_col}{min_row}:{max_col}{max_row}")
        


    for row in ws.iter_rows():
        for cell in row:
            #rounding
            #print(cell.value)
            if type(cell.value) == float:
                cell.value = round(cell.value, 3)
            new_cell = new_ws.cell(row=cell.row, column=cell.column, value=cell.value)
                #print(cell.value)

    #Hiding gridlines     
    new_ws.sheet_view.showGridLines = False
    #smazani automaticky generovaného sheetu:
    if counter == 1:
        del new_wb['Sheet']
    # Save the new workbook to disk
    new_filename = f"{adresa_tabs}\\formated\\to_format_.xlsx"
    #print(dir(new_wb.save())
    new_wb.save(new_filename)
    
    # Launch the new spreadsheet
    #os.startfile(new_filename)

    #break

['brand_m_11']
['brand_m_11', 'brand_m_12']
['brand_m_11', 'brand_m_12', 'brand_m_13']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15', 'brand_m_16']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15', 'brand_m_16', 'brand_m_17']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15', 'brand_m_16', 'brand_m_17', 'brand_m_18']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15', 'brand_m_16', 'brand_m_17', 'brand_m_18', 'brand_m_19']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15', 'brand_m_16', 'brand_m_17', 'brand_m_18', 'brand_m_19', 'brand_m_20']
['brand_m_11', 'brand_m_12', 'brand_m_13', 'brand_m_14', 'brand_m_15', 'brand_m_16', 'brand_m_17', 'brand_m_18', 'brand_m_19', 'brand_m_20', 'brand_m_21']
['c:\\Users\\jan.petr\\Projekty\\2021\\BHT\\vyjezdy\\BLG\\brand_t

### Nacte tabulky, zvormatuje a spoji
    #Finální verze, která je rychlejší: i zformatuje tabulky:

In [ ]:
adresa_tabs = r"c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m"
input_tabs = f"{adresa_tabs}\\formated\\to_format_.xlsx"
formated_tabs = f"{adresa_tabs}\\formated\\formated_.xlsx"
output_tabs = f"{adresa_tabs}\\formated\\combined_.xlsx"




def find_coord_formated_tabs (ws):
    counter = 0
    for row in ws.iter_rows():
        counter += 1
        for cell in row:
            if cell.fill.start_color.index == 22:
                return counter

with xw.App(visible=False) as app:
    wb_xlwing = xw.Book(input_tabs)   
    macro_vba_formattabs_beta = app.macro("'c:\\Users\\jan.petr\\Provozni\\excel_tab_copying\\CTables 4.2.35.xlsm'!a_FORMATTABS_beta")
    #macro_vba_sig_95 = app.macro("'c:\\Users\\jan.petr\\Provozni\\excel_tab_copying\\CTables 4.2.35.xlsm'!b_sig_min_run")
    for sheet in wb_xlwing.sheets:
        sheet.activate()
        macro_vba_formattabs_beta()
    wb_xlwing.save(formated_tabs)
    wb_xlwing.close()

wb = openpyxl.load_workbook(formated_tabs)

with xw.App(visible=False) as app:
    counter = 0
    width_list = []
    wb_xlwing = xw.Book(formated_tabs)
    for filename in wb.sheetnames:
        counter += 1
        #print(filename)
        ws = wb[filename]
        ws_xlwing=wb_xlwing.sheets[filename]
        if counter == 1:
            max_row = find_coord_formated_tabs (ws)
            freezed = ws.freeze_panes
        for col in range(ws.min_column, ws.max_column + 1):
            width_to_append = ws_xlwing.range(str(get_column_letter(col)+"1")).column_width
            if counter == 1:
                width_list.append(width_to_append)
            else:
                if width_list[col - 1] < width_to_append:
                    width_list[col - 1] = width_to_append


#print(width_list)

with xw.App(visible=False) as app:
    wb_init = xw.Book(formated_tabs)
    wb_res = xw.Book()
    ws_res = wb_res.sheets[0]

    for index,ws_init in enumerate (wb_init.sheets):
        if index == 0:
            ws_init.used_range.copy()
        else:
            ws_init.used_range[max_row - 1:,:].copy()
        ws_res.used_range[-1:,:].offset(row_offset=1).paste()
    #ws_res["1:1"].delete() # This is just to delete the first row, which is empty.
    for col in range(ws.min_column, ws.max_column + 1):
        ws_res.range(str(get_column_letter(col)+"1")).column_width = width_list[col-1]
    #přizplsobí výšku bunek
    ws_res.autofit(axis="rows")
    #uloži
    wb_res.save(output_tabs)
    wb_res.close(); wb_init.close()
    

#Ukotveni pricek
new_wb = openpyxl.load_workbook(output_tabs)
new_ws = new_wb.active
#https://stackoverflow.com/questions/48752276/trying-to-unfreeze-panes-with-openpyxl-causes-damages
new_ws.sheet_view.topLeftCell = 'A1'
new_ws.freeze_panes = freezed
#print(new_ws.freeze_panes)

#Hiding gridlines
new_ws.sheet_view.showGridLines = False

new_wb.save(output_tabs)

c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_11.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_12.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_13.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_14.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_15.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_16.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_17.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_18.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_19.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_20.XLSX
c:\Users\jan.petr\Projekty\2021\BHT\vyjezdy\BLG\brand_tabs_m\brand_m_21.XLSX
